# Prelude

In [ ]:
# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:\n")
!git status --short

# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from bgcellmodels.common import analysis
from bgcellmodels.mechanisms import noise, synapses

import neuron; h = neuron.h
import bluepyopt.ephys as ephys

# Create cell

In [ ]:
# Load cell model
from bgcellmodels.models.striatum.Mahon2000_MSN import mahon_pynn_model
icell = h.MahonMSN()
icell.setparams_corbit2016()
soma = icell.soma[0]

# CTX Afferents

## Required Data

- number of synapses
- presynaptic firing rate + pattern
- magnitude of EPSC / EPSP
- strength and timescale of STD

## Experimental Data

### Number & Location

- 10.000 asymmetrical synapses per MSN
    + usually correspond to excitatory synapses
    + source: *Guzman (2003)*

- each cortico-striatal axon makes only a single or a few contacts with a single MSN
    + source: *Steiner, Tseng (2017) - Ch. 1*

### Synaptic Response

- 8.3 mV = magnitude of first EPSP in D2 MSN upon stimulation of CTX -> STR afferents
    + !!! cortical stimulation between layers V-VI with electrode => likely many tracts activated
        - let's stay you manage to stimulate 10% of the 5000 afferents
        - then this is the response to 500 simultaneously activated synapses
    + with holding potential of -86 mV
    + source: *Ding (2012)*
    
    
- 0.59 = fraction of EPSP_5 / EPSP_2 when stimulating at 50 Hz
    + source: *Ding (2012)*

### Firing Pattern

- 1-5 Hz during relaxed neutral posture (Bergman book chapter)
    + Poisson-like discharge rate with a slight tendency toward bursting. 

## Calibration Results

- following parameters yields good match to EPSPs in Ding (2010):
    + `gmax_AMPA = 0.6e-4`
    + `tau_r = 1.0`, `tau_d = 4.0`
    + `tau_rec = 100`, `tau_facil = 100`, `P_release = 0.35`


- In Corbit (2016) paper the synaptic conductances are
    + `g_NaF = 35` (35 mS in XPP -> 0.035 S/cm2 in NEURON with area 100 um2)
        - `0.035 mA/cm2 * 1e-6 [cm2] = 0.035 * 1e-6 mA`
    + `gsyn_MSN-MSN = 0.14` (0.14 mS in XPP)
        - TM synapses expressed in `uS` so correspond to `1e-6 [S*mV = mA]`
        - `0.00014 = 1.4e-4 uS`
    + `gsyn_FSI-MSN = 0.15`
    + `gpas_CTX-MSN = 0.066 (DNORM) and 0.083 (DD)`

## CTX->STR.MSN (AMPA)

In [ ]:
# Calibrate AMPA synapse to get desired PSP time course

# Save inputs
stim_data = {}
istim = None

def make_ctx_afferents(num_pre=1, rate=50, noise=0, num_spikes=1e9, tstart=750, 
                       gsyn_base=6e-5, scale_gsyn=1.0, intraburst_rate=0, interburst_interval=200, 
                       burst_duration=50.0, burst_noise=1.0):
    # Clear previous inputs
    stim_data['CTX'] = {}
    stim_data['CTX']['synapses'] = []
    stim_data['CTX']['netstims'] = []
    stim_data['CTX']['netcons'] = []
    
    # Make new inputs
    for i in range(num_pre):
        seg = soma(0.5)
        syn = h.GLUsyn(seg)
        stim_data['CTX']['synapses'].append(syn)
        
        # One spike generator for background spikes
        stim = h.NetStim()
        stim.interval = rate**-1*1e3
        stim.number = num_spikes
        stim.noise = noise
        stim.start = tstart
        nc = h.NetCon(stim, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        stim_data['CTX']['netstims'].append(stim)
        stim_data['CTX']['netcons'].append(nc)
        
        # One spike generator for bursts (does not spike between bursts)
        if intraburst_rate != 0:
            bstim = h.BurstStim()
            bstim.fast_invl = intraburst_rate ** -1 * 1e3
            bstim.slow_invl = interburst_interval
            bstim.burst_len = burst_duration * 1e-3 * intraburst_rate
            bstim.start = tstart
            bstim.noise = burst_noise
            bnc = h.NetCon(bstim, syn)
            bnc.delay = 1.0
            bnc.weight[0] = 1
            stim_data['CTX']['netstims'].append(bstim)
            stim_data['CTX']['netcons'].append(bnc)
        
        # Common STP parameters
        syn.tau_rec = 100.0
        syn.tau_facil = 100.0
        syn.U1 = P_release = 0.35 # release probability
        
        # AMPA synapse parameters
        syn.gmax_AMPA = ctx_gAMPA = np.random.normal(gsyn_base * scale_gsyn, 1.1e-5)
        syn.tau_r_AMPA = 1.0
        syn.tau_d_AMPA = 4.0

        # NMDA synapse parameters
        syn.gmax_NMDA = 0.25 * ctx_gAMPA # conductance_nS * peak_factor * 1e-3
        syn.tau_r_NMDA = 3.7
        syn.tau_d_NMDA = 80

In [ ]:
make_ctx_afferents(num_pre=10, rate=200, noise=0, num_spikes=5)

# Prevent spiking: hyperpolarizing current (like in article figure)
if istim is None:
    istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = 0.0 # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'synGLU': stim_data['CTX']['synapses'][0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    # Synapse variables
    'gAMPA': {'pointp':'synGLU', 'var':'g_AMPA'},
    'gNMDA': {'pointp':'synGLU', 'var':'g_NMDA'},
    'iAMPA': {'pointp':'synGLU', 'var':'i_AMPA'},
    'iNMDA': {'pointp':'synGLU', 'var':'i_NMDA'},
    'iGLU': {'pointp':'synGLU', 'var':'i'},
    'Rrp': {'pointp':'synGLU', 'var':'R'},
    'Use': {'pointp':'synGLU', 'var':'Use'},
}

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -77.4 # from Mahon article code
h.tstop = 2000.0
h.init()
h.run()
# nrnsim.run(h.tstop, h.dt)

In [ ]:
# Plot recorded traces
plot_interval = (700, 1000)

plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.ylim((-90, -70))
plt.xlim(plot_interval)
plt.grid(True)

In [ ]:
# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['gAMPA'].as_numpy(), 'b', label='gAMPA')
ax.plot(t_soma, vec_dict['gNMDA'].as_numpy(), 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.set_xlim(plot_interval)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['iAMPA'].as_numpy(), 'b', label='iAMPA')
ax.plot(t_soma, vec_dict['iNMDA'].as_numpy(), 'r', label='iNMDA')
ax.plot(t_soma, vec_dict['iGLU'].as_numpy(), 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.set_xlim(plot_interval)
ax.legend()

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True, timeRange=plot_interval) # yRange=(-80,40), timeRange=

## CTX->STR.MSN (NMDA)

No data -> use ~0.25 NMDA/AMPA ratio for gsyn

# MSN Afferents

## Required Data

- number of synapses
- presynaptic firing rate + pattern
- magnitude of IPSC / IPSP
- strength and timescale of STD

## Experimental Data

### Number & Location

- 2.500 symmetrical synapses per MSN
    + can be DAergic, ACHergic, or GABAergic
    + 13% of these are DAergic
    + 13% of these are cholinergic (extrapolation, no data)
    + => 74% = __1.850 synapses are GABAergic__


- 2.700 candidate pre-synaptic MSN can contact dendritic tree of target MSN
    + 10% of neighboring MSN are connected to eachother
    + => 0.1 * 2.700 = __270 MSN afferents__ to MSN (afferent colaterals)
    + => 5 * 270 = __1.350 contacts from those 270 MSN afferents__

### Synaptic Response

IPSPs from MSN collaterals are very weak, and occur distaly

- see Tepper, Wilson [2008](https://www.sciencedirect.com/science/article/pii/S0165017307002275)
    + conclusion and figures
    + __EPSP amptlide__ is in range (0.05, 0.35) mV depending on synapse location


- Taverna, Surmeier et al. (2008)
    + __no significant depression__ but fairly high failure rate of transmission
        - see Fig. 5A shows ~50 Hz D2->D2 MSN stimulation and resulting IPSPs (polarity reversed because of solution used): 
    + Fig. 2C : median rise time : __tau_r = 0.75 ms__ for D2-D2
    + Fig. 2C : median decay time : __tay_d = 12.5 ms__ for D2-D2

### Firing Pattern

- 0-1 spikes per second (Bergman book chapter) with possible bursts/plateaus
    + lower than cortex
    + Poisson-like discharge rate with a slight tendency toward bursting. 

- See Steiner, Tseng (2017) handbook - Chapters 1.II-1.V, 20, 22

## Calibration Results

- `gmax_GABAA = 0.75e-5` yields IPSP of ~0.35 mV with cell at plateau of ~ -40 mV

## MSN->MSN (GABAA)

In [ ]:
# Calibrate GABAA conductance so we get PSP of desired magnitude
stim_data = {}
istim = None

def make_msn_afferents(num_pre=1, rate=50, noise=0, num_spikes=1e9, tstart=750, 
                       scale_gsyn=1.0, intraburst_rate=0, interburst_interval=200, 
                       burst_duration=50.0, burst_noise=1.0):
    # Clear previous inputs
    stim_data['MSN'] = {}
    stim_data['MSN']['synapses'] = []
    stim_data['MSN']['netstims'] = []
    stim_data['MSN']['netcons'] = []
    
    for i in range(num_pre):
        seg = soma(0.5)
        syn = h.GABAsyn(seg)
        stim_data['MSN']['synapses'].append(syn)
        
        # One spike generator for background spikes
        stim = h.NetStim()
        stim.interval = rate ** -1 * 1e3
        stim.number = num_spikes
        stim.noise = noise
        stim.start = tstart
        nc = h.NetCon(stim, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        stim_data['MSN']['netstims'].append(stim)
        stim_data['MSN']['netcons'].append(nc)
        
        # One spike generator for bursts (does not spike between bursts)
        if intraburst_rate != 0:
            bstim = h.BurstStim()
            bstim.fast_invl = intraburst_rate ** -1 * 1e3
            bstim.slow_invl = interburst_interval
            bstim.burst_len = burst_duration * 1e-3 * intraburst_rate
            bstim.start = tstart
            bstim.noise = burst_noise
            bnc = h.NetCon(bstim, syn)
            bnc.delay = 1.0
            bnc.weight[0] = 1
            stim_data['MSN']['netstims'].append(bstim)
            stim_data['MSN']['netcons'].append(bnc)
        
        # Common STP parameters
        syn.tau_rec = 20.0
        syn.tau_facil = 0.5
        syn.U1 = P_release = 0.8 # release probability

        # AMPA synapse parameters
        syn.gmax_GABAA = 7.714e-5 * scale_gsyn # [uS] or set netcon weight in [nS]
        syn.tau_r_GABAA = 0.75
        syn.tau_d_GABAA = 12.5

        # NMDA synapse parameters
        syn.gmax_GABAB = 0.0 # conductance_nS * peak_factor * 1e-3
        syn.tau_r_GABAB = 5.0
        syn.tau_d_GABAB = 80

make_msn_afferents(1, rate=50, noise=0, num_spikes=10)

In [ ]:
# Bring cell in state where we can see effect of PSPs

# make sure cell is depolarized
if istim is None:
    istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = 0.005 # 100 pA = 0.1 nA

# remove NaF channels
for seg in soma:
    seg.gnabar_Nam = 0.0

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'synGABA': stim_data['MSN']['synapses'][0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -77.4 # from Mahon article code
h.tstop = 2000.0
h.init()
h.run()

In [ ]:
# Plot recorded traces
plot_interval = (700, 1000)

plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
# plt.ylim((-90, -70))
plt.xlim(plot_interval)
plt.grid(True)

In [ ]:
# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['gAMPA'].as_numpy(), 'b', label='gAMPA')
ax.plot(t_soma, vec_dict['gNMDA'].as_numpy(), 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.set_xlim(plot_interval)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['iAMPA'].as_numpy(), 'b', label='iAMPA')
ax.plot(t_soma, vec_dict['iNMDA'].as_numpy(), 'r', label='iNMDA')
ax.plot(t_soma, vec_dict['iGLU'].as_numpy(), 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.set_xlim(plot_interval)
ax.legend()

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True, timeRange=plot_interval) # yRange=(-80,40), timeRange=

# Interneuron Afferents



## Experimental Data

### Number & Location

- Interneuron connections:
    - 1.850 - 1.350 = __500 GABAergic contacts from other inhibitory sources__
    - 30 interneurons converge on single MSN (rough calculation)
    - __4-27 interneurons converge on single MSN__ (experimental data)
    - 8-15 contacts per interneuron (both FS and LTS)
    - => 30 x 15 = 500 contacts per MSN coming from interneurons
    - 16 neurons of a single class converging on an MSN (FS, LTS, Ach)
    - => __16 x 15 = 240 contacts per MSN coming from one interneuron class__

### Synaptic Response

- TODO: see TepperWilson and sources mentioned in Corbit (2016) e.g. in table


- Gittis (2010), Fig. 5 & Fig. 6
    + Fig. 6: IPSC depresses to about 25% of initial amplitude after ~7 spikes @ 50 hz
        - 30% for 10 hz, 20% for 100 Hz
    + Fig. 5: tau_r = 0.5 ms
    + Fig. 5: tau_d = 8.0 +/- 3.2 ms (mean +/- std)


- Gittis (2011)
    + connection probability FS -> MSN is doubled after DA depletion
    + i.e. MSN have twice as many FS inputs (from twice the amount of FS neurons)


- Tepper, Wilson (2007)
    + Fig. 2: PSPs of 0.75 - 4 mV caused by single AP of FS interneuron
    + Fig. 2: failure rate is < 1% (change to trigger PSP)
    + Fig. 2: PSP delays or even abolishes MSN spikes

### Firing Pattern

- Gittis (2010)
    + table 1: max firing rate is 286 +/- 88 Hz
    + Methods: Inhibitory frequencies were based on the assumption that:
        - any given FS interneuron operates across a __dynamic range of 10 - 100 Hz__ (Berke et al., 2004; Berke, 2008)
    + Results: Recordings from the striatum of awake behaving mice reveal that FS interneurons express __average firing rates of 10 - 15 Hz__ and __modulate their firing rates up to 60 - 80 Hz__ during behavioral tasks (Berke et al., 2004; Berke, 2008)


- Tepper, Wilson (2007)
    + The neurons exhibit a very hyperpolarized membrane potential in vitro and do not fire spontaneously. When stimulated with a series of increasing amplitude depolarizing pulses, the neurons do not fire at all below a certain threshold. Over this threshold a tiny increment in stimulus amplitude results in a short burst of a few spikes. Further increases lead to episodes of high frequency firing interspersed with silent periods. Sufficiently strong stimuli evoke sustained firing at rates > 200 Hz (Kawaguchi, 1993; Kawaguchi et al., 1995; Koós and Tepper, 1999, 002; Bracci et al., 2002; Plotkin et al., 2005; Taverna et al., 2007)

## Calibration Results

- following parameters yields good match to PSPs in Tepper (2007) Fig. 2 and STP in Gittis (2010) Fig. 6:
    + `gmax_GABA = [1.0e-5, 5.0e-5]` for PSPs of [0.75, 4] mV
    + `tau_r = 0.5`, `tau_d = 8.0`
    + `tau_rec = 200`, `tau_facil = 0.5`, `P_release = 0.35`


## FSI -> MSN (GABAA)

In [ ]:
# Calibrate GABAA conductance so we get PSP of desired magnitude

stim_data = {}
istim = None

def make_fsi_afferents(num_pre=1, rate=50, noise=0, num_spikes=1e9, tstart=750, 
                       scale_gsyn=1.0, intraburst_rate=0, interburst_interval=200, 
                       burst_duration=50.0, burst_noise=1.0):
    # Clear previous inputs
    stim_data['FSI'] = {}
    stim_data['FSI']['synapses'] = []
    stim_data['FSI']['netstims'] = []
    stim_data['FSI']['netcons'] = []
    
    # Make new inputs
    for i in range(num_pre):
        seg = soma(0.5)
        syn = h.GABAsyn(seg)
        stim_data['FSI']['synapses'].append(syn)
        
        # One spike generator for background spikes
        stim = h.NetStim()
        stim.interval = rate ** -1 * 1e3
        stim.number = num_spikes
        stim.noise = noise
        stim.start = tstart
        nc = h.NetCon(stim, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        stim_data['FSI']['netstims'].append(stim)
        stim_data['FSI']['netcons'].append(nc)
        
        # One spike generator for bursts (does not spike between bursts)
        if intraburst_rate != 0:
            bstim = h.BurstStim()
            bstim.fast_invl = intraburst_rate ** -1 * 1e3
            bstim.slow_invl = interburst_interval
            bstim.burst_len = burst_duration * 1e-3 * intraburst_rate
            bstim.start = tstart
            bstim.noise = burst_noise
            bnc = h.NetCon(bstim, syn)
            bnc.delay = 1.0
            bnc.weight[0] = 1
            stim_data['FSI']['netstims'].append(bstim)
            stim_data['FSI']['netcons'].append(bnc)

        # Common STP parameters
        syn.tau_rec = 200.0
        syn.tau_facil = 0.5
        syn.U1 = P_release = 0.35 # release probability

        # AMPA synapse parameters
        syn.gmax_GABAA = 1.286e-4 * scale_gsyn # [uS] or set netcon weight in [nS]
        syn.tau_r_GABAA = 0.5
        syn.tau_d_GABAA = 8.0

        # NMDA synapse parameters
        syn.gmax_GABAB = 0.0 # conductance_nS * peak_factor * 1e-3
        syn.tau_r_GABAB = 5.0
        syn.tau_d_GABAB = 80


make_fsi_afferents(1, rate=50, noise=0, num_spikes=10)

In [ ]:
# Bring cell in state where we can see effect of PSPs

# make sure cell is depolarized
if istim is None:
    istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = 0.005 # 100 pA = 0.1 nA

# remove NaF channels
# for seg in soma:
#     seg.gnabar_Nam = 0.0

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'synGABA': stim_data['FSI']['synapses'][0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    # Synapse variables
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synGABA', 'var':'g_GABAB'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
    'iGABAA': {'pointp':'synGABA', 'var':'i_GABAA'},
    'iGABAB': {'pointp':'synGABA', 'var':'i_GABAB'},
    'iGABA': {'pointp':'synGABA', 'var':'i'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -77.4 # from Mahon article code
h.tstop = 2000.0
h.init()
h.run()

In [ ]:
# Plot recorded traces
plot_interval = (700, 1000)

plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
#plt.ylim((-41, -55))
plt.xlim(plot_interval)
plt.grid(True)

In [ ]:
# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['gGABAA'].as_numpy(), 'b', label='gGABA-A')
ax.plot(t_soma, vec_dict['gGABAB'].as_numpy(), 'r', label='gGABA-B')
ax.set_ylabel('conductance (uS)')
ax.set_xlim(plot_interval)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['iGABAA'].as_numpy(), 'b', label='iGABA-A')
ax.plot(t_soma, vec_dict['iGABAB'].as_numpy(), 'r', label='iGABA-B')
ax.plot(t_soma, vec_dict['iGABA'].as_numpy(), 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.set_xlim(plot_interval)
ax.legend()

# Plot vesicle pool dynamcis
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['Rrp'].as_numpy(), 'b', label='Rrp')
ax.plot(t_soma, vec_dict['Use'].as_numpy(), 'r', label='Use')
ax.set_xlim(plot_interval)
ax.grid(True)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['gGABAA'].as_numpy(), 'g', label='gGABA-A')
ax.set_xlim(plot_interval)
ax.grid(True)
ax.legend()

# All Inputs

The Observations Database contains following observations that may help in configuring the distribution of synapses on the STN dendritic tree:

## Experimental Data

- Afferents and prsynaptic firing rate firing rate
- Postsynaptic firing rate
- Ratio of inputs

### Guzman (2003)

Article p. 7:

> In voltage-clamp mode, unitary IPSCs range from 20 to 250 pA (Koos et al., 2002); the minimum is in the range of quantal events in other synapses (Hanse and Gustafsson, 2001). With our stimulation parameters, the average synaptic current, 67 pA (10-250 pA), was in the unitary range. There- fore, an average of five boutons per connection is a reasonable number (quantum, 10-20 pA, depending on intracellular Cs+). Similar stimulation parameters yielded an average IPSC of 93 pA (20-500 pA) for intrastriatal inhibition, suggesting that axon collateral inhibition is quantitatively less than interneuron inhibition; 

- Average of five boutons per afferent onto MSN

> however, a spiny neuron receives  10,000 asymmetrical (Kincaid et al., 1998) and  2500 symmetrical synapses (Ingham et al., 1998). Symmetrical synapses can be dopaminergic, cholinergic, or GABAergic, with dopaminergic being  13% (Roberts et al., 2002). Assuming that cholinergic inputs are a similar percentage, 650 symmetrical synapses per spiny cell are not GABAergic. This leaves 1850 GABAergic synapses per spiny cell. How many come from axon collaterals? There are  2840 striatal neurons inside the volume of a spiny dendritic tree (Oorschot, 1996; Kincaid et al., 1998). Most spiny neurons have their axon collaterals restricted to this volume (Kawaguchi et al., 1990). If 95% of the neurons are spiny, the number of potentially contacting neurons onto a single centered spiny cell is  2700. Only  10% of neighboring spiny neurons are connected with one another (Czubayko and Plenz, 2002; Tunstall et al., 2002); therefore, connecting spiny neurons total only  270. An average of five contacts yields 1350 inhibitory synapses per spiny cell arising from axon collaterals: two-thirds of the GABAergic inputs.

- 10.000 asymmetrical synapses per MSN
    + (usually correspond to excitatory synapses)


- 2.500 symmetrical synapses per MSN
    + can be DAergic, ACHergic, or GABAergic
    + 13% of these are DAergic
    + 13% of these are cholinergic (extrapolation, no data)
    + => 74% = 1.850 synapses are GABAergic


- 2.700 candidate pre-synaptic MSN can contact dendritic tree of target MSN
    + 10% of neighboring MSN are connected to eachother
    + => 0.1 * 2.700 = 270 MSN afferents to MSN (afferent colaterals)
    + => 5 * 270 = 1.350 contacts (boutons) from those 270 MSN afferents
    + => 1.850 - 1.350 = 500 GABAergic contacts/synapses come from other inhibitory sources

> A similar calculation for interneurons explains the remaining contacts: 5% of the surrounding 2840 striatal neurons are interneurons, which is  140. One-fourth of the surrounding spiny cells receive inputs from a centered interneuron (Koos and Tepper, 1999), which leaves  30 interneurons converging in a single spiny cell. Fast- and low-threshold spiking interneurons might leave up to 15 (range, 8–15) contacts per spiny cell (Kubota and Kawaguchi, 2000). Thus, 30 x 15 = 500 contacts from interneurons. The later calculation has experimental support: 4-27 FS interneurons may converge on a single projection cell (Koos and Tepper, 1999). This makes an average of 16 interneurons of a single class converging on a spiny cell, which makes 16 x 15 = 240 contacts from one class of interneuron. Assuming the same number of contacts from both GABAergic interneuron genres (Kubota and Kawaguchi, 2000), the number of interneuronal contacts is again  500. Thus, 70% of all GABAergic contacts are from axon collaterals, and 30% are from interneurons.

- __NOTE: if connection probability (same in both directions per definition) is X%, then the number of inputs on a post-synaptic cell is X% * N_PRE and the number of targets of a pre-synaptic cell is X% * N_POST__
- 30 interneurons converge on single MSN (rough estimate)
- 4-27 interneurons converge on single MSN (experimental data)
- 8-15 contacts per interneuron (both FS and LTS)
- => 30 x 15 = 500 contacts per MSN coming from interneurons
- 16 neurons of a single class converging on an MSN (FS, LTS, Ach)
- => 16 x 15 = 240 contacts per MSN coming from one interneuron class

### Steiner, Tseng (2017)

- 2,800,000 = number of MSN in rat, unilaterally
- 17,000,000 = nuber of cortico-striatal projection neurons in rat, unilaterally
- number of interneurons in rat, unilaterally
    + 21,300 GABAergic SOM/NPY/NOS expressing interneurons
    + 16,900 GABAergic PV expressing interneurons
    + 13,200 GABAergic Calretinin expressing interneurons
    + 12,200 ACHergic interneurons
- 46,000 = number of GPe _projection_ neurons in rat, unilaterally

### Corbit (2017)

- 40 MSN cells
- 8 FSI cells
- 8 GPe cells

| Pre | Post | P_conn       | N_pre (N*P_conn) | gsyn (mS/cm2)    | tau_r | tau_d |
|-----|------|--------------|------------------|------------------|-------|-------|
| FSI | MSN  | 37.5 (DD:75) | 3 (DD: 6)        | 0.12 (code:0.15) | 0.5   | 4.35  |
| MSN | MSN  | 35           | 14               | 0.09 (code:0.14) | 0.5   | 10.0  |
| GPe | MSN  | 37.5         | 0 (Fig.4: 3)     | 0.003 (DD:0.01)  | 0.5   | 8.3   |
| CTX | MSN  | 100.0        | 1 (passive)      | 0.066 (DD:0.083) | 0     | 0     |

## Parameter Calculations

Each model synapse will represent all the contacts by a single afferent cell.

- MSN->MSN
    + 270 MSN afferents * 5 contacts/afferent = __1.350 contacts__ per MSN coming __from other MSN__
    + => __270 model synapses from MSN__


- IN->MSN:
    - 500 GABAergic contacts _not_ coming from MSNs
    - 4-27 (experimental) or 30 (calculated) interneurons converge on single MSN
    - 16 neurons of a single class converging on an MSN (FS, LTS, Ach)
    - 8-15 contacts per interneuron (both FS and LTS)
    - => 30 x 15 = __500 contacts__ per MSN coming __from interneurons__
    - => 16 x 15 = __240 contacts__ per MSN coming __from one interneuron class__ (FS/LTS/Ach)
    - => __16 model synapses from FS in CONTROL__ (GABAergic)
    - => __32 model synapses from FS in DD__
    - => __16 model synapses from LTS__ (GABAergic)
    - => __16 model synapses from ACH__ (ACHergic)


- CTX->MSN
    - 10.000 asymmetrical synapses per MSN (*Guzman (2003)*)
        + usually correspond to excitatory synapses
        + => __10.000 contacts coming from CTX__
    - each cortico-striatal axon makes only a single or a few contacts with a single MSN
        + source: *Steiner, Tseng (2017) - Ch. 1*
        + => __10.000 / 2 = 5.000 model synapses from CTX__
    - no two striatal MSN share common inputs from the cortex
        + this would imply we need NUM_MSN * 5000 independent spike trains

### Ratio of contacts

The ratios of inputs for `CTX:MSN:FS` with real population sizes and spike rates is:
- in __CONTROL__ - 5000:270:16
- in __DD__ - 5000:270:32

### Ratio of total conductance

We can calculate it ourselves by multiplying unitary response magnitudes with input sizes:
- `*16 [FSI] : *270 [MSN] : 28pA*X [GPE] : *5000 [CTX]`
- TODO: lookup Imax in pA

Or according to *Corbit (2016)*
- FSI : MSN : GPE
- = `0.15*3 [FSI] : 0.09*14 [MSN] : 0.003*3 [GPE] : 0.066*1 [CTX]`
- = `0.45 [FSI] : 1.26 [MSN] : 0.009 [GPE] : 0.066 [CTX]` 

Now we can use following procedure:
- `gtot_i` = multiply realistic unitary conductance in our model (realistic PSP) by actual number of contacts
- `gtot_j` = multiply by ratio of total conductances with another input type
- `gsyn_j` = divide by simulated number of contacts for that input type

### Result

We are constrainted by the total conductances and the number of pre-synaptic inputs of each type. For setting the number of pre-synaptic inputs we are constrained by our chosen population sizes and the conservation of connection  patterns (i.e. converge/divergence, connection probabilities): `N_in_j = P_conn * N_pop_j`. An unrealistic number of shared inputs will connect the network too tightly or loosely. The population size itself is also dependent on the connection pattern in order to conserve number of shared inputs.


- contacts from 10% of its neighboring MSN => __10 model synapses from MSN__
    + 270/10 = 27 => __`gsyn_msn` scale factor = 27.0__
- if we make 20 FSI and follow the observed 25% connection probability between neighboring MSN-IN:
    + => 20 * 0.25 = __5 model synapses from MSI__
    + => 16/5 = 3.2 => __`gsyn_fs` scale factor = 3.2__
- we are constrained by the fact that no two striatal MSN share common inputs from cortex
    + if we use N independent spike trains at K times the rate of CTX the reduction factor is `5000 / (K*N)` 
    + => __`gsyn_ctx` scale factor = 5000/(K*N)__

## ALL -> MSN (GABA + GLU)

In [ ]:
# Clear old inputs
stim_data = {}
istim = None

# Parameters for afferents
afferent_params = aff_par = {}

aff_par['CTX'] = {}
aff_par['CTX']['nsyn'] = NUM_PRE_CTX = 20
aff_par['CTX']['rate_scale'] = SCALE_RATE_CTX = 5
aff_par['CTX']['rate'] = RATE_CTX = (5.0 - 1.0) / 2 * SCALE_RATE_CTX

aff_par['MSN'] = {}
aff_par['MSN']['nsyn'] = NUM_PRE_MSN = 14
aff_par['MSN']['rate'] = RATE_MSN = (1.0 - 0.0) / 2

aff_par['FSI'] = {}
aff_par['FSI']['nsyn'] = NUM_PRE_FSI = 3
aff_par['FSI']['rate'] = RATE_FSI = 12.5 # 10-15 during rest, 60-80 task-related


make_ctx_afferents(NUM_PRE_CTX, rate=RATE_CTX,
                   noise=1.0, tstart=20,
                   intraburst_rate=100, interburst_interval=500.0, 
                   burst_duration=50.0, burst_noise=1.0)

make_msn_afferents(NUM_PRE_MSN, rate=RATE_MSN, noise=1.0, tstart=20)

make_fsi_afferents(NUM_PRE_FSI, rate=RATE_FSI, 
                   noise=1.0, tstart=20,
                   intraburst_rate=70.0, interburst_interval=500.0, 
                   burst_duration=50.0, burst_noise=1.0)



# Check that we have correct number of inputs
for pre in stim_data.keys():
    print("Afferents from {pre} have:\n\t- {nsyn} synapses"
          "\n\t- {nstim} spike generators".format(
              pre=pre, nsyn=len(stim_data[pre]['synapses']),
              nstim=len(stim_data[pre]['netstims'])))

In [ ]:
# Activity-dependent scaling of synaptic weights
from bgcellmodels.mechanisms import plasticity

# Clear old references
hpwa = None
rec_secs = None

hpwa = h.VanRossumHPWA(soma(0.5))
hpwa.scaling = 0
hpwa.sensing = 0
hpwa.activitytau = 2e3 # sense activity over window of last ~ 1 second
hpwa.activitybeta = 5e-1 # (s^-1 Hz^-1) Scaling strength proportional to activity error (was 4e-5 in article)
hpwa.set_target_rate(1.0)

# Add weights to scale
for syn in stim_data['CTX']['synapses']:
    h.setpointer(syn._ref_gmax_AMPA, 'temp_wref', hpwa)
    hpwa.add_wref(1)
    h.setpointer(syn._ref_gmax_NMDA, 'temp_wref', hpwa)
    hpwa.add_wref(1)

# Set up activity sensing
spike_rec = h.NetCon(soma(0.5)._ref_v, hpwa, sec=soma)
spike_rec.threshold = 0.0
spike_rec.delay = 1
spike_rec.weight[0] = 1

# Enable plasticity after delay
cvode = h.CVode()
def enable_sensing():
    hpwa.sensing = 1
    print("Start sensing at time {}".format(h.t))
def enable_scaling():
    hpwa.scaling = 1
    print("Start scaling at time {}".format(h.t))
def add_events():
    cvode.event(10.0, enable_sensing)
    cvode.event(500.0, enable_scaling)
fih = h.FInitializeHandler(add_events)

## Record & Simulate

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'soma_spiker': h.NetCon(soma(0.5)._ref_v, None, -10.0, 0, 0),
    'hpwa': hpwa,
#     'prox': proximal_seg,
#     'mid': middle_seg,
#     'dist': distal_seg,
#     'synGABA': synapses[0],
}

trace_specs = {
    't_global': {'var': 't'},
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'AP_soma': {'netcon': 'soma_spiker'},
    # HPWA weight scaling
    'a_HPWA': {'pointp':'hpwa', 'var':'activity'},
    'scale_HPWA': {'pointp':'hpwa', 'var':'scalefactor'},
    # Synapse variables (pointp added below)
    'gAMPA': {'pointp':'synCTX0', 'var':'g_AMPA'},
    'gNMDA': {'pointp':'synCTX0', 'var':'g_NMDA'},
    'iAMPA': {'pointp':'synCTX0', 'var':'i_AMPA'},
    'iNMDA': {'pointp':'synCTX0', 'var':'i_NMDA'},
    'iGLU':  {'pointp':'synCTX0', 'var':'i'},
#     'Rrp': {'pointp':'synGLU', 'var':'R'},
#     'Use': {'pointp':'synGLU', 'var':'Use'},
    'gGABAA': {'pointp':'synFSI0', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synFSI0', 'var':'g_GABAB'},
    'iGABAA': {'pointp':'synFSI0', 'var':'i_GABAA'},
    'iGABAB': {'pointp':'synFSI0', 'var':'i_GABAB'},
    'iGABA':  {'pointp':'synFSI0', 'var':'i'},
    'Gprot':  {'pointp':'synFSI0', 'var':'G'},
}

# Record ALL spikes and synaptic currents
for pop in 'CTX', 'MSN', 'FSI':
    for i_syn, nc in enumerate(stim_data[pop]['netcons']):
        con_label = "con{}{}".format(pop, i_syn)
        syn_label = "syn{}{}".format(pop, i_syn)
        rec_secs[con_label] = nc
        rec_secs[syn_label] = nc.syn()
        trace_specs['AP_'+con_label] = {'netcon': con_label}
        trace_specs['i_'+syn_label] = {'pointp': syn_label, 'var':'i'}

# SETPARAM: low sampling interval for slow traces
rec_dt = 5.0 # 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -77.4 # from Mahon article code
h.tstop = 60e3
# h.init()

import time
tstart = time.time()
h.run()

tstop = time.time()
cputime = tstop - tstart
num_segments = sum((sec.nseg for sec in h.allsec()))
print("Simulated {} segments for {} ms in {} ms CPU time".format(
        num_segments, h.tstop, cputime))

## Plot & Analyze

### Plot Weight Scaling

In [ ]:
# Plot GLU synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6), sharex=True)

# Signals
activity_hz = vec_dict['a_HPWA'].as_numpy() * 1e3
scale_vec = vec_dict['scale_HPWA'].as_numpy()
t_rec = np.arange(len(activity_hz)) * rec_dt

# Plotted interval
interval = (2000, h.tstop)
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

ax = axes[0]

ax.plot(t_rec, activity_hz, 'b', label='activity')
ax.vlines(vec_dict['AP_soma'].as_numpy(), 
        activity_hz.min(), 0.2*activity_hz.max(),
        color='red', linewidth=1.0)
ax.set_xlim((interval))
ax.set_ylabel('activity (Hz)')
ax.legend()
ax.grid(True)
ax.set_title('HPWA activity sensor')

ax = axes[1]
ax.plot(t_rec, scale_vec, 'g', label='scale')
ax.set_xlim((interval))
ax.set_ylabel('scale_factor')
ax.legend()
ax.grid(True)
ax.set_title('HPWA weight scale factor')


print("Final scale factor is: {}".format(hpwa.scalefactor))
print("Final gmax_AMPA is {}".format(stim_data['CTX']['synapses'][1].gmax_AMPA))
print("Final gmax_NMDA is {}".format(stim_data['CTX']['synapses'][1].gmax_NMDA))

### Plot Voltage Traces

In [ ]:
# Plot recorded traces
# t_global = vec_dict.pop('t_global').as_numpy() # inconsistent if dt != rec_dt 
interval = (0, 2000.0)

# Plot somatic voltage traces
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt
# plt.figure()
# plt.plot(t_rec, v_soma)
# plt.grid(True)
spikes_soma = vec_dict['AP_soma'].as_numpy()
spike_range = (spikes_soma > interval[0]) & (spikes_soma <= interval[1])
soma_rate = len(spikes_soma[spike_range]) / float(interval[1]-interval[0]) * 1e3
analysis.plotTraces(vec_dict, rec_dt, interval, includeTraces=['V_soma'], title="Soma @ {} Hz".format(soma_rate))

### Plot Spike Trains

In [ ]:
# Plot spikes
spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP_conCTX'))
analysis.plotRaster(spike_data, interval, color='b', title='{} CTX spiketrains'.format(len(spike_data)))

spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP_conMSN'))
analysis.plotRaster(spike_data, interval, color='r', title='{} MSN spiketrains'.format(len(spike_data)))

spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP_conFSI'))
analysis.plotRaster(spike_data, interval, color='r', title='{} FSI spiketrains'.format(len(spike_data)))

### Plot Synaptic Currents

In [ ]:
# Plot synaptic variables
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

# Plot GLU synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gAMPA'].as_numpy()[a:b], 'b', label='gAMPA')
ax.plot(t_plot, vec_dict['gNMDA'].as_numpy()[a:b], 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GLU Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iAMPA'].as_numpy()[a:b], 'b', label='iAMPA')
ax.plot(t_plot, vec_dict['iNMDA'].as_numpy()[a:b], 'r', label='iNMDA')
ax.plot(t_plot, vec_dict['iGLU'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.legend()
ax.grid(True)

In [ ]:
# Plot GABA synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gGABAA'].as_numpy()[a:b], 'b', label='gGABAA')
ax.plot(t_plot, vec_dict['gGABAB'].as_numpy()[a:b], 'r', label='gGABAB')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GABA Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iGABAA'].as_numpy()[a:b], 'b', label='iGABAA')
ax.plot(t_plot, vec_dict['iGABAB'].as_numpy()[a:b], 'r', label='iGABAB')
ax.plot(t_plot, vec_dict['iGABA'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.grid(True)
ax.legend()

In [ ]:
# G protein concentration
# fig, ax = plt.subplots(1, 1, figsize=(10,4))
# ax.plot(t_plot, vec_dict['Gprot'].as_numpy()[a:b], 'b', label='Gprot')
# ax.set_ylabel('G protein')
# ax.legend()
# ax.grid(True)
# ax.set_title('G protein concentration')

# Plot total GABA synapse current
itot_GABA = vec_dict['i_synFSI0'].as_numpy()
for i in range(1, len(stim_data['FSI']['synapses'])):
    itot_GABA += vec_dict['i_synFSI{}'.format(i)].as_numpy()
for i in range(0, len(stim_data['MSN']['synapses'])):
    itot_GABA += vec_dict['i_synMSN{}'.format(i)].as_numpy()

itot_GLU = vec_dict['i_synCTX0'].as_numpy() * -1
for i in range(1, len(stim_data['CTX']['synapses'])):
    itot_GLU -= vec_dict['i_synCTX{}'.format(i)].as_numpy()

plt.figure(figsize=(10,4))
plt.plot(t_rec, itot_GABA, 'r', label='i_GABA')
plt.plot(t_rec, itot_GLU, 'b', label='i_GLU')
plt.ylabel('current (nA)')
plt.xlabel('time (ms)')
plt.legend()
plt.grid()
plt.show()
plt.suptitle('Total GABA and GLU current')
# Plot remaining traces
# figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),